In [1]:
import argparse
import json
import math
import uuid
from pathlib import Path
import pandas as pd
import adaptive
import ghalton
import nest_asyncio
import numpy as np
from openmc_model import find_tbr_hcpb
from plot_sampling_coordinates import make_3d_plot
from tqdm import tqdm

adaptive.notebook_extension()
nest_asyncio.apply()


from openmc_model import find_tbr_hcpb

/home/jshim/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
/home/jshim/anaconda3/lib/python3.8/site-packages/adaptive/notebook_integration.py:33: RuntimeWarning: holoviews is not installed; plotting is disabled.
  warnings.warn(


In [2]:
number_of_simulations = 16
results = []

In [ ]:
print("running simulations with random sampling")

for i in tqdm(range(number_of_simulations)):

    breeder_percent_in_breeder_plus_multiplier_ratio = np.random.uniform(0, 100)
    blanket_breeder_li6_enrichment = np.random.uniform(1, 100)

    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "random"
    results.append(result)

make_3d_plot(df[df['sample']=='random'])

  0%|          | 0/16 [00:00<?, ?it/s]

running simulations with random sampling


  6%|▋         | 1/16 [00:10<02:32, 10.16s/it]

In [ ]:
print("running simulations with grid sampling")

number_of_steps = math.floor(math.sqrt(number_of_simulations))

for breeder_percent_in_breeder_plus_multiplier_ratio in tqdm(np.linspace(0, 100, number_of_steps)):
    for blanket_breeder_li6_enrichment in np.linspace(0, 100, number_of_steps):

        result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                               blanket_breeder_li6_enrichment)

        result["sample"] = "grid"
        results.append(result)

make_3d_plot(df[df['sample']=='grid'])

In [ ]:
print("running simulations with halton sampling")

sequencer = ghalton.Halton(2)
coords = sequencer.get(number_of_simulations)

for coord in tqdm(coords):

    breeder_percent_in_breeder_plus_multiplier_ratio = coord[0] * 100  # scales sampling from 0 to 100
    blanket_breeder_li6_enrichment = coord[1] * 100  # scales sampling from 0 to 100

    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "halton"
    results.append(result)

make_3d_plot(df[df['sample']=='halton'])

In [ ]:
print("running simulations with adaptive sampling")

def find_tbr(x):

    breeder_percent_in_breeder_plus_multiplier_ratio, blanket_breeder_li6_enrichment = x
                           
    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "adaptive"
    results.append(result)


    return result["tbr"]

learner = adaptive.Learner2D(find_tbr, bounds=[(0, 100), (0, 100)])

runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.npoints > number_of_simulations)

runner.ioloop.run_until_complete(runner.task)

make_3d_plot(df[df['sample']=='adaptive'])